# IP21 and Alarm data analyse

Author: Mark Chinnock, Anova Data Solutions   
Date: July 2021   
## Content

Introduction

    1.1 Background

    1.2 Problem

    1.3 Audience

    Data
    Methodology
    Results
    Discussion
    Conclusion



## Introduction

IP21 data is a generic name given to data recorded from machine sensors on each of the manufacturing lines.  The sensors provide readings of (but not exclusively): temperature, humidity, pressure, and counters.   
<p>The readings come from the machines at regular timed intervals (time series data) regardless of the current reading/performance from the sensor – ie it is not alerting of a specific scenario/reading.  Each sensor is documented on a detailed schematic and named so that it can be mapped to a plant/line/machine/sensor type.
<p>There are also alerts/alarms that are issued from the manufacturing lines.  The alarms have also been named to indicate specific areas / machines.  Although often mentioned along with IP21, there is no direct correlation between the alerts/alarms being issued into one database and the IP21 data being issued into another database. There would need to be a complex mapping exercise to link specific alerts/alarms to related IP21 data.


In [1]:
import pandas as pd
import numpy as np
import datetime

# this dir should point at where ever your sanofi Mclaren directory is syncing
# dir = "C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Race Dashboard data/"

from sys import platform
# if platform == "linux" or platform == "linux2":
    # linux    
if platform == "darwin":
    # OS X
    dir = "/Users/mark/Documents/GitHub/IP21/"
elif platform == "win32":
    # Windows...
    # dir = "C:/Users/mark_/Documents/McLaren2021/Sanofi/IP21/"
    # trying the microsoft teams sanofi share folder
    dir = "C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Frankfurt sprint/SFD/OEE data/"

    


In [2]:
def get_col_widths(dataframe):
    # First we find the maximum length of the index column   
    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    # Then, we concatenate this to the max of the lengths of column name and its values for each column, left to right
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]


In [3]:
# read Detailed Events file
file = (dir + 'AL6_Packaging_Detailed_Events.xlsx')
df = pd.read_excel(file, parse_dates=True, index_col='Start Date')

# read in OEE losses
file = (dir + 'AL6_OEE_Losses.xlsx')
df_OEE = pd.read_excel(file)
df_OEE.rename(columns=({'Calendar Day':'Date', 
                        'Point of OEE':'OEE',
                        'Loss Category':'Category'}), inplace=True)
df_OEE['OEE'] = pd.to_numeric(df_OEE['OEE'], errors='coerce')
df_OEE['OEE'] = df_OEE['OEE'].mul(100)
df_OEE = df_OEE.set_index('Date')
df_OEE.shape

(2741, 2)

get rid of any pre April 2021 data - wasn't valid for AL6

In [4]:
start_date = '2021-04-01'
end_date = datetime.date.today()
df_OEE = df_OEE[df_OEE.index >= start_date].sort_values('Date')
df_OEE.Category.unique()

array(['Qualifizierung/Validierung (11)', 'Production', 'Minor Loss',
       'Without cause', 'Sonstiges und Unbekannte (26)',
       'Störungen und Fehlerbehebung (25)', 'Speed Loss',
       'W&I (geplante Linie) (23)',
       'Organisatorische Verluste und Wartezeiten (22)',
       'Rüstvorgänge (21)', 'Ungeplante Linie (14)',
       'Feier/Brückentag (12)', 'Autonome Wartungen (24)',
       'vorbeugende Instandhaltung/Wartung (13)', 'Sonstige (15)',
       'Quality Loss'], dtype=object)

Let's translate these categories!

In [5]:
df_OEE['Category'].replace({
'Qualifizierung/Validierung (11)':'Qualification/validation', 
'Sonstiges und Unbekannte (26)':'Other and unknown',
'Störungen und Fehlerbehebung (25)':'Malfunctions and troubleshooting', 
'W&I (geplante Linie) (23)':'W&I (planned line)',
'Organisatorische Verluste und Wartezeiten (22)':'Organizational losses and waiting times',
'Rüstvorgänge (21)':'Changeover', 
'Ungeplante Linie (14)':'Unplanned Line',
'Feier/Brückentag (12)':'Celebration / long day', 
'Autonome Wartungen (24)':'Autonomous maintenance',
'vorbeugende Instandhaltung/Wartung (13)':'Preventive maintenance / servicing', 
'Sonstige (15)':'Other'}, inplace=True)

In [6]:
# do I need this?  

# set up date range with min start_date and max start_date recorded earlier
daterange = pd.date_range(start = start_date, end = end_date, freq='D')

pd.DataFrame({'Date': daterange.date})

# df = pd.merge(df1, df, on=['Date','UI'], how='outer')

,Date
0,2021-04-01
1,2021-04-02
2,2021-04-03
3,2021-04-04
4,2021-04-05
...,...
108,2021-07-18
109,2021-07-19
110,2021-07-20
111,2021-07-21


In [7]:
df_OEE['Month'] = df_OEE.index.strftime("%B")
df_OEE['Day'] = df_OEE.index.strftime("%d")
df_OEE.head()

# df_OEE.groupby(['Day', 'Month', 'Category'])[['OEE']].sum()

df_OEE['Month'] = df_OEE['Month'].astype(pd.api.types.CategoricalDtype(categories=['April','May','June', 'July'], ordered=True))


production = df_OEE[df_OEE.Category == ('Production')]
changeover = df_OEE[df_OEE.Category == ('Changeover')]

pivot = production.pivot(index='Day', columns='Month', values='OEE').style.background_gradient(cmap='Oranges_r')\
       # .format("{:.2f}")

pivot

Month,April,May,June,July
Day,,,,
01,43.382099,43.102778,54.364583,44.023148
02,16.429006,38.669444,21.286574,31.937310
03,61.401389,29.590123,44.448400,17.029861
04,0.008333,30.714903,80.177778,0.000000
05,nan,32.486574,53.335463,nan
06,27.228704,33.511179,18.668519,nan
07,33.662500,67.476157,29.443308,nan
08,50.584259,60.335648,39.222029,nan
09,49.161111,41.127778,25.174102,51.367983


In [8]:
pivot.to_excel(dir + "AL6_production_losses_pivot.xlsx")

In [9]:
category_order = ['Production',
'Organizational losses and waiting times', 
'Changeover', 
'Malfunctions and troubleshooting',
'Other and unknown', 
'Without cause', 
'Minor Loss', 
'Speed Loss', 
'Autonomous maintenance',
'Celebration / long day', 
'Other',
'Preventive maintenance / servicing', 
'Quality Loss',
'Unplanned Line',
'W&I (planned line)',
'Qualification/validation']

not using this one at the moment, as the categories are all over the place

In [22]:
writer = pd.ExcelWriter(dir + 'AL6_OEE_Losses_pivot.xlsx', engine='xlsxwriter')
for i in df_OEE.Month.unique():
    pivot_OEE = df_OEE.pivot(index='Day', columns=['Category','Month'], values='OEE').filter(regex=i)
    styler_OEE = pivot_OEE.style.background_gradient(cmap='Oranges')\
        .background_gradient(subset=['Production'], cmap='Oranges_r')\
    
    styler_OEE.to_excel(writer, sheet_name=i)
    worksheet = writer.sheets[i]
    workbook = writer.book
    cell_format = workbook.add_format()
    decimal = workbook.add_format({'num_format': '0.00'})
#     worksheet.set_column('A:H', 45, cell_format)
#     call function get_col_widths to auto size the excel columns
    for x, width in enumerate(get_col_widths(pivot_OEE)):
        #set each column width to be 1*25 that of the max string so that it fits nicely
        worksheet.set_column(x, x, width * 1.25)    



workbook = writer.book
workbook.add_format()
workbook.add_format({'num_format': '0.00'})
writer.save()

This sorts the pivot by category, then month.

In [18]:
writer = pd.ExcelWriter(dir + 'AL6_OEE_Losses_pivot.xlsx', engine='xlsxwriter')

pivot_OEE_sorted = df_OEE.pivot(index='Day', columns=['Category','Month'], values='OEE')
pivot_OEE_sorted.columns = pivot_OEE_sorted.columns.set_levels(
    pd.CategoricalIndex(
        pivot_OEE_sorted.columns.levels[0],
        ['Production',
        'Organizational losses and waiting times', 
        'Changeover', 
        'Malfunctions and troubleshooting',
        'Other and unknown', 
        'Without cause', 
        'Minor Loss', 
        'Speed Loss', 
        'Autonomous maintenance',
        'W&I (planned line)',
        'Celebration / long day', 
        'Other',
        'Preventive maintenance / servicing', 
        'Quality Loss',
        'Unplanned Line',
        'Qualification/validation'],
        ordered=True
    ), level=0
)

pivot_OEE_sorted = pivot_OEE_sorted.sort_index(axis=1).style.background_gradient(cmap='Oranges')\
    .background_gradient(subset=['Production'], cmap='Purples')

pivot_OEE_sorted.to_excel(writer)
workbook = writer.book
writer.save()

In [15]:
df_OEE.groupby(['Category','Month']).describe().transpose()

Category  Autonomous maintenance                                   \
Month                      April        May       June       July   
OEE count              29.000000  29.000000  26.000000  15.000000   
    mean                2.200777   2.992478   3.742103   2.113928   
    std                 4.408781   4.039306   5.025886   4.397251   
    min                 0.000000   0.000000   0.000000   0.000000   
    25%                 0.000000   0.000000   0.502025   0.000000   
    50%                 0.620370   1.012346   3.007995   0.726852   
    75%                 2.942130   4.481481   4.711516   1.332392   
    max                22.726852  15.724537  23.834491  16.607762   

Category  Celebration / long day           Changeover                        \
Month                      April  May June      April        May       June   
OEE count                    5.0  1.0  1.0  29.000000  29.000000  26.000000   
    mean                     0.0  0.0  0.0  10.866925   8.049728   6.767113   
    std                      0.0  NaN  NaN  18.301047   6.980477  10.173623   
    min                      0.0  0.0  0.0   0.000000   0.000000   0.000000   
    25%                      0.0  0.0  0.0   2.824074   1.851852   0.000000   
    50%                      0.0  0.0  0.0   7.143519   7.131944   3.182870   
    75%                      0.0  0.0  0.0  12.169753  11.748868  10.366030   
    max                      0.0  0.0  0.0  99.995370  29.032407  47.500000   

Category   ... Unplanned Line       W&I (planned line)                       \
Month      ...           June  July              April  May       June July   
OEE count  ...           26.0  15.0               12.0  8.0  12.000000  9.0   
    mean   ...            0.0   0.0                0.0  0.0   0.560957  0.0   
    std    ...            0.0   0.0                0.0  0.0   1.310115  0.0   
    min    ...            0.0   0.0                0.0  0.0   0.000000  0.0   
    25%    ...            0.0   0.0                0.0  0.0   0.000000  0.0   
    50%    ...            0.0   0.0                0.0  0.0   0.000000  0.0   
    75%    ...            0.0   0.0                0.0  0.0   0.000000  0.0   
    max    ...            0.0   0.0                0.0  0.0   3.371528  0.0   

Category  Without cause                                   
Month             April        May       June       July  
OEE count     29.000000  29.000000  26.000000  15.000000  
    mean       4.234435   7.735997   1.904903   1.585354  
    std        3.924070   7.918484   1.657682   1.479806  
    min        0.000000   0.000000   0.000000   0.000000  
    25%        1.516204   4.101852   0.527199   0.177662  
    50%        2.003472   6.299769   1.712269   1.416667  
    75%        6.394676   9.224993   3.304398   2.313601  
    max       12.229167  42.574074   5.504630   4.671296  

[8 rows x 62 columns]

In [14]:
pivot_OEE = df_OEE.pivot(index='Day', columns=['Category','Month'], values='OEE')
pivot_OEE.stack()

Category   Autonomous maintenance  Celebration / long day  Changeover  \
Day Month                                                               
01  April                0.000000                     0.0   11.976852   
    May                  0.312500                     NaN    4.842593   
    June                 0.619213                     NaN    0.000000   
    July                 7.040509                     NaN    0.000000   
02  April                0.000000                     0.0   18.625000   
...                           ...                     ...         ...   
29  June                 1.216992                     NaN   11.150019   
30  April                0.000000                     NaN    7.019676   
    May                  0.000000                     NaN   29.032407   
    June                10.797222                     NaN    7.166667   
31  May                  1.012346                     NaN    0.000000   

Category   Malfunctions and troubleshooting  Minor Loss  \
Day Month                                                 
01  April                         22.481481   12.870370   
    May                           21.302083   23.060185   
    June                           2.065972   10.186343   
    July                          22.467593    8.652778   
02  April                         21.027778   14.275463   
...                                     ...         ...   
29  June                           9.029851    9.446996   
30  April                         11.686343   16.143519   
    May                            0.000000    7.166667   
    June                          14.008333    7.388889   
31  May                           34.214506    6.540123   

Category   Organizational losses and waiting times  Other  Other and unknown  \
Day Month                                                                      
01  April                                 3.421296    NaN          10.745370   
    May                                   0.701389    0.0           6.754630   
    June                                  1.945602    0.0           7.738426   
    July                                  3.653935    0.0           5.106481   
02  April                                 2.925926    NaN           9.358796   
...                                            ...    ...                ...   
29  June                                  9.169537    0.0           6.174895   
30  April                                 7.013889    NaN          10.497685   
    May                                  20.972222    NaN           7.490741   
    June                                  8.333333    0.0           0.725000   
31  May                                   0.000000    NaN           4.195988   

Category   Preventive maintenance / servicing  Production  \
Day Month                                                   
01  April                                 NaN   43.382099   
    May                                   0.0   43.102778   
    June                                  NaN   54.364583   
    July                                  NaN   44.023148   
02  April                                 0.0   16.429006   
...                                       ...         ...   
29  June                                  NaN   40.484561   
30  April                                 0.0   47.721528   
    May                                   NaN   26.219444   
    June                                  NaN   30.341111   
31  May                                   NaN   36.513889   

Category   Qualification/validation  Quality Loss  Speed Loss  Unplanned Line  \
Day Month                                                                       
01  April                       0.0           NaN   -6.983951             0.0   
    May                         0.0           NaN  -10.181481             0.0   
    June                        0.0           NaN   20.543981             0.0   
    July                        0.0       

In [15]:
df_grouped = df.groupby([pd.Grouper(freq='D'), 'Machine'])['Duration (min.)'].sum().reset_index()
pivot = df_grouped.pivot(index='Start Date', columns='Machine', values='Duration (min.)')
pivot.fillna(0, inplace=True)
pivot['Total Duration'] = pivot.sum(axis=1)
pivot.nlargest(15, 'Total Duration').style.background_gradient(cmap='Purples')


Machine,AL6 Endverpacker,AL6 FS Bedruckung,AL6 Kartonierer,AL6 Kontrollwaage,Total Duration
Start Date,,,,,
2021-04-10 00:00:00,6.550000,19.516667,1151.666667,0.000000,1177.733333
2021-05-04 00:00:00,19.066667,6.400000,786.683333,0.000000,812.150000
2021-04-12 00:00:00,2.250000,0.000000,725.416667,0.000000,727.666667
2021-04-09 00:00:00,90.033333,82.783333,479.166667,0.000000,651.983333
2021-04-27 00:00:00,36.500000,21.700000,511.900000,0.000000,570.100000
2021-05-29 00:00:00,31.533333,124.483333,314.933333,0.000000,470.950000
2021-04-29 00:00:00,24.850000,88.633333,352.800000,0.000000,466.283333
2021-05-06 00:00:00,45.316667,57.433333,360.250000,0.000000,463.000000
2021-05-05 00:00:00,34.233333,39.916667,382.533333,0.000000,456.683333


In [16]:
df.loc['2021-06-07'].groupby(['Machine','Stoppage Reason 3','Last Level Reason']).sum().sort_values('Duration (min.)', ascending=False)

Duration (min.)
Machine           Stoppage Reason 3                 Last Level Reason                                       
AL6 Kartonierer   Minor Loss                        -                                              84.833333
                  Without cause                     Without cause                                  19.416667
                  Störungen und Fehlerbehebung (25) Robatec (251441304)                            16.750000
AL6 FS Bedruckung Störungen und Fehlerbehebung (25) Preisetiketten Spendeaggregat (2512604)        11.566667
AL6 Endverpacker  Störungen und Fehlerbehebung (25) Kartonmagazin (251270504)                       9.650000
AL6 Kartonierer   Störungen und Fehlerbehebung (25) F2_224- Abholwerkzeug PENs (251440803)          7.583333
AL6 FS Bedruckung Minor Loss                        -                                               0.083333

In [17]:
df[df.Machine.str.contains('Kart')].loc['2021-06-07'].sort_values('Duration (min.)', ascending=False)

,Shift Long Name,End Date,Duration (min.),Stoppage Reason 3,Last Level Reason,Machine,Comments,Creator
Start Date,,,,,,,,
2021-06-07 08:25:00,FRUE - 07/06/2021 06:00:00,2021-06-07 08:36:23,11.383333,Störungen und Fehlerbehebung (25),Robatec (251441304),AL6 Kartonierer,-,DE021051
2021-06-07 19:02:22,SPAE - 07/06/2021 14:00:00,2021-06-07 19:09:57,7.583333,Störungen und Fehlerbehebung (25),F2_224- Abholwerkzeug PENs (251440803),AL6 Kartonierer,-,DE018876
2021-06-07 21:32:24,SPAE - 07/06/2021 14:00:00,2021-06-07 21:37:58,5.566667,Without cause,Without cause,AL6 Kartonierer,-,OEE_INTEGRATOR
2021-06-07 07:54:38,FRUE - 07/06/2021 06:00:00,2021-06-07 08:00:00,5.366667,Störungen und Fehlerbehebung (25),Robatec (251441304),AL6 Kartonierer,-,OEE_INTEGRATOR
2021-06-07 20:59:09,SPAE - 07/06/2021 14:00:00,2021-06-07 21:01:35,2.433333,Without cause,Without cause,AL6 Kartonierer,-,OEE_INTEGRATOR
...,...,...,...,...,...,...,...,...
2021-06-07 10:54:03,FRUE - 07/06/2021 06:00:00,2021-06-07 10:54:09,0.100000,Minor Loss,-,AL6 Kartonierer,-,OEE_INTEGRATOR
2021-06-07 12:15:00,FRUE - 07/06/2021 06:00:00,2021-06-07 12:15:05,0.083333,Minor Loss,-,AL6 Kartonierer,-,OEE_INTEGRATOR
2021-06-07 10:30:10,FRUE - 07/06/2021 06:00:00,2021-06-07 10:30:12,0.033333,Minor Loss,-,AL6 Kartonierer,-,OEE_INTEGRATOR


In [21]:
df['Shift Long Name'].str[:4].unique()

array(['FRUE', 'SPAE', 'NACH'], dtype=object)